In [1]:
import pandas as pd 
import numpy as np 
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.express as px
import sys 

In [18]:
import os 
plants = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\Plant_data_halfhour.csv")
whole_time=[pd.to_datetime(day+' '+hour) for day,hour in zip(plants['Date'],plants['Time'])]
plants['time']=whole_time
plants['pixel diff']=plants['Area per plant'].diff(1)
plants=plants.set_index('time')
plants['time'] = plants.index
#plants = plants[:-1]
re_arranged_plants = pd.DataFrame({})

enviros = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\dataframe4.csv")


df_collection = []
        
for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])

    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass

#变成当天光期和当天暗期的整体对比
#15号光期指的是14号18点到15号11点
#15号暗期指的是15号11点到15号18点

In [12]:
# Define a function that generates a list of time ranges for a period
def generate_time_ranges(start_time, end_time, own_freq, other_freq):
    time_ranges = []
    while start_time < end_time:
        time_ranges.append((start_time, start_time + own_freq))
        start_time += own_freq + other_freq
    return time_ranges

# Define the start and end times and frequency for each period
light_start_time = pd.Timestamp('2023-05-4 18:00:00')
light_end_time = pd.Timestamp('2023-05-15 11:00:00')
light_frequency = pd.Timedelta(hours=17)
dark_start_time = pd.Timestamp('2023-05-4 11:00:00')
dark_end_time = pd.Timestamp('2023-05-15 18:00:00')
dark_frequency = pd.Timedelta(hours=7)

#需要补充光质设定的信息，等查博消息

# Generate the time ranges for each period
light_time_ranges = generate_time_ranges(light_start_time, light_end_time, light_frequency,dark_frequency)
dark_time_ranges = generate_time_ranges(dark_start_time, dark_end_time, dark_frequency, light_frequency)

In [13]:
light_time_ranges

[(Timestamp('2023-05-04 18:00:00'), Timestamp('2023-05-05 11:00:00')),
 (Timestamp('2023-05-05 18:00:00'), Timestamp('2023-05-06 11:00:00')),
 (Timestamp('2023-05-06 18:00:00'), Timestamp('2023-05-07 11:00:00')),
 (Timestamp('2023-05-07 18:00:00'), Timestamp('2023-05-08 11:00:00')),
 (Timestamp('2023-05-08 18:00:00'), Timestamp('2023-05-09 11:00:00')),
 (Timestamp('2023-05-09 18:00:00'), Timestamp('2023-05-10 11:00:00')),
 (Timestamp('2023-05-10 18:00:00'), Timestamp('2023-05-11 11:00:00')),
 (Timestamp('2023-05-11 18:00:00'), Timestamp('2023-05-12 11:00:00')),
 (Timestamp('2023-05-12 18:00:00'), Timestamp('2023-05-13 11:00:00')),
 (Timestamp('2023-05-13 18:00:00'), Timestamp('2023-05-14 11:00:00')),
 (Timestamp('2023-05-14 18:00:00'), Timestamp('2023-05-15 11:00:00'))]

In [15]:
plants.keys()

Index(['Date', 'Time', 'Group', 'Pixel', 'Area', 'Plant', 'Area per plant',
       'No.', 'pixel diff', 'time'],
      dtype='object')

In [27]:
a,b = plants[['Pixel','Area']].mean()

In [28]:
plants['Pixel'].mean()

578192.1160747664

In [39]:
enviros['time'] = pd.to_datetime(enviros['time'])
plants['time'] = pd.to_datetime(plants['time'])

In [41]:
df = plants
for time_range in light_time_ranges:
    #time_range = pd.to_datetime(time_range)
    for day in pd.unique(df['time'].dt.date):
        df_day = df[df['time'].dt.date == day]
        enviros_day = enviros[enviros['time'].dt.date == day]
        df_light = df_day[(df_day['time'] >= time_range[0]) & (df_day['time'] < time_range[1])]
        enviro_light = enviros_day[(enviros_day['time'] >= time_range[0]) & (enviros_day['time'] < time_range[1])]


In [44]:
pixel_diffs, Ts, RHs, blues, reds, greens, CO2_s, ECs, pHs = 9 * [ ]
pixel_diffs

ValueError: not enough values to unpack (expected 9, got 0)

In [109]:
time_range = light_time_ranges[2]
print(time_range)
day = pd.unique(df['time'].dt.date)[0]
print(day)
enviros[enviros['time'].dt.date == day]

(Timestamp('2023-05-06 18:00:00'), Timestamp('2023-05-07 11:00:00'))
2023-04-11


,Unnamed: 0,空调光期温度设定,空调暗期温度设定,空调温度上限设定,空调温度下限设定,新风机手动开度,新风机手自动模式,空调自动模式,户外温度,户外湿度,...,空调开机指令下发,空调手动模式选择,空调风速选择,空调手动温度设定,空调手动操作指令下发,空调手自动切换,电磁阀手自动选择,电磁阀手动开阀,电磁阀手动关阀,time
3646,3646,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:00:10.296437
3647,3647,22.000000,18.0,19.5,18.5,0.0,False,True,17.526043,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:01:11.284056
3648,3648,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:02:12.148145
3649,3649,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:03:13.120894
3650,3650,22.000000,18.0,19.5,18.5,0.0,False,True,17.526043,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:04:14.026903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5037,5037,23.700001,18.0,19.5,18.5,100.0,False,True,14.117477,100.0,...,False,1.0,4.0,23.700001,False,False,False,False,False,2023-04-11 23:55:43.903372
5038,5038,23.600000,18.0,19.5,18.5,100.0,False,True,14.166668,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:56:44.991006
5039,5039,23.600000,18.0,19.5,18.5,100.0,False,True,14.068287,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:57:45.991275
5040,5040,23.600000,18.0,19.5,18.5,100.0,False,True,14.068287,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:58:47.056068


In [110]:
#大整体，假设大家光质没区别   

df = plants

def time_range_gather(df,enviros, time_ranges):
    days = []
    pixel_diffs = []
    Ts = []
    RHs = []
    blues = []
    reds = []
    greens = []
    CO2_s = []
    ECs = []
    pHs = []
    # Iterate over the periods and time ranges and extract the data for each time range separately
    for time_range in time_ranges:
        print(time_range)
        #time_range = pd.to_datetime(time_range)
        for day in pd.unique(df['time'].dt.date):
            #print(time_range)
            start_date = pd.to_datetime(time_range[0]).date()
            end_date = pd.to_datetime(time_range[1]).date()
            if day >= start_date and day <= end_date:
                print(day)
                df_day = df[df['time'].dt.date == day]
                enviros_day = enviros[enviros['time'].dt.date == day]
                df_light = df_day[(df_day['time'] >= time_range[0]) & (df_day['time'] < time_range[1])]
                enviro_light = enviros_day[(enviros_day['time'] >= time_range[0]) & (enviros_day['time'] < time_range[1])]
                days.append(day)
                pixel_diffs.append(df_light['pixel diff'].mean())

                print(enviros_day)
                print(enviro_light)
                
                T, RH, blue, red, green, co2, ec, ph = enviro_light[['2号室内温度','2号室内湿度','1号蓝比','1号红比','1号绿比','1号室内CO2浓度','营养液EC','营养液PH']].mean()
                Ts.append(T)
                RHs.append(RH)
                blues.append(blue)
                reds.append(red)
                greens.append(green)
                CO2_s.append(co2)
                ECs.append(ec)
                pHs.append(ph)        

    return days,pixel_diffs, Ts, RHs, blues, reds, greens, CO2_s, ECs, pHs

days_light,pixel_diffs_light,Ts_light, RHs_light, blues_light, reds_light, greens_light, CO2_s_light, ECs_light, pHs_light = time_range_gather(df,enviros,light_time_ranges)

(Timestamp('2023-05-04 18:00:00'), Timestamp('2023-05-05 11:00:00'))
2023-05-04
2023-05-04 18:00:00
Empty DataFrame
Columns: [Unnamed: 0, 空调光期温度设定, 空调暗期温度设定, 空调温度上限设定, 空调温度下限设定, 新风机手动开度, 新风机手自动模式, 空调自动模式, 户外温度, 户外湿度, 1号室内温度, 1号室内湿度, 1号室内CO2浓度, 2号室内温度, 2号室内湿度, 2号室内CO2浓度, 1号PPFD, 1号色温, 1号PAR, 1号红比, 1号绿比, 1号蓝比, 2号PPFD, 2号色温, 2号PAR, 2号红比, 2号绿比, 2号蓝比, 3号PPFD, 3号色温, 3号PAR, 3号红比, 3号绿比, 3号蓝比, 1号风速, 2号风速, 营养液EC, 营养液PH, 营养液液温, 空调手动开关机, 空调开机指令下发, 空调手动模式选择, 空调风速选择, 空调手动温度设定, 空调手动操作指令下发, 空调手自动切换, 电磁阀手自动选择, 电磁阀手动开阀, 电磁阀手动关阀, time]
Index: []

[0 rows x 50 columns]
Empty DataFrame
Columns: [Unnamed: 0, 空调光期温度设定, 空调暗期温度设定, 空调温度上限设定, 空调温度下限设定, 新风机手动开度, 新风机手自动模式, 空调自动模式, 户外温度, 户外湿度, 1号室内温度, 1号室内湿度, 1号室内CO2浓度, 2号室内温度, 2号室内湿度, 2号室内CO2浓度, 1号PPFD, 1号色温, 1号PAR, 1号红比, 1号绿比, 1号蓝比, 2号PPFD, 2号色温, 2号PAR, 2号红比, 2号绿比, 2号蓝比, 3号PPFD, 3号色温, 3号PAR, 3号红比, 3号绿比, 3号蓝比, 1号风速, 2号风速, 营养液EC, 营养液PH, 营养液液温, 空调手动开关机, 空调开机指令下发, 空调手动模式选择, 空调风速选择, 空调手动温度设定, 空调手动操作指令下发, 空调手自动切换, 电磁阀手自动选择, 电磁阀手动开阀, 电磁阀手动关阀, time]
Index: []

[0 rows x 

In [96]:
enviros_day['time'],time_range

(10719   2023-05-15 08:14:03.399347
 10720   2023-05-15 08:15:04.711344
 10721   2023-05-15 08:16:06.102268
 10722   2023-05-15 08:17:07.423242
 10723   2023-05-15 08:18:09.125196
                    ...            
 10904   2023-05-15 11:25:16.684238
 10905   2023-05-15 11:26:17.723965
 10906   2023-05-15 11:27:18.643012
 10907   2023-05-15 11:28:19.592977
 10908   2023-05-15 11:29:20.644671
 Name: time, Length: 190, dtype: datetime64[ns],
 (Timestamp('2023-05-14 18:00:00'), Timestamp('2023-05-15 11:00:00')))

In [92]:
pixel_diffs_light,Ts_light

([-0.344444444444447,
  -18.64222222222222,
  0.05534653465346533,
  0.0015972222222222251,
  0.06777777777777778,
  0.0030718954248365937,
  0.0714814814814815,
  -0.004577464788732409,
  0.019578947368421085,
  0.040144927536231875,
  0.09166666666666666,
  0.005833333333333357,
  0.006509433962264331,
  0.10034246575342465],
 [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  24.286335370555427])

In [56]:
df['pixel diff']

time
2023-04-11 20:38:16      NaN
2023-04-11 20:38:26   -19.59
2023-04-11 20:38:38    39.64
2023-04-11 20:38:50   -29.69
2023-04-11 20:39:01    15.67
                       ...  
2023-05-15 08:45:55    39.65
2023-05-15 08:46:16   -18.51
2023-05-15 08:46:36    -7.07
2023-05-15 08:46:57     3.90
2023-05-15 08:47:18     6.10
Name: pixel diff, Length: 5350, dtype: float64

In [3]:
plants
for i in range(len(plants))

,Date,Time,Group,Pixel,Area,Plant,Area per plant,No.,pixel diff,time
time,,,,,,,,,,
2023-04-11 20:38:16,2023/4/11,20:38:16,2-lower,914767,1533.19,12,127.77,0,NaN,2023-04-11 20:38:16
2023-04-11 20:38:26,2023/4/11,20:38:26,2-mid,719009,1190.03,11,108.18,1,-19.59,2023-04-11 20:38:26
2023-04-11 20:38:38,2023/4/11,20:38:38,1-upper,827799,1478.21,10,147.82,2,39.64,2023-04-11 20:38:38
2023-04-11 20:38:50,2023/4/11,20:38:50,2-upper,727688,1299.44,11,118.13,3,-29.69,2023-04-11 20:38:50
2023-04-11 20:39:01,2023/4/11,20:39:01,3-lower,788481,1338.05,10,133.80,4,15.67,2023-04-11 20:39:01
...,...,...,...,...,...,...,...,...,...,...
2023-05-15 08:45:55,2023-05-15,08:45:55,3-lower,477366,810.09,10,81.01,4,39.65,2023-05-15 08:45:55
2023-05-15 08:46:16,2023-05-15,08:46:16,3-mid,447563,750.02,12,62.50,5,-18.51,2023-05-15 08:46:16
2023-05-15 08:46:36,2023-05-15,08:46:36,3-upper,258179,498.91,9,55.43,7,-7.07,2023-05-15 08:46:36


In [2]:

df_collection = []

for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])

    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass
dataframes4 = {}
csv_path = "arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)
print(num_files)
for i in range(num_files):
    dataframes4['df'+str(i)] = pd.read_csv("arranged_data4/sub_df"+str(i)+".csv")

combine_df = pd.concat([dataframes4[key] for key in dataframes4.keys()],ignore_index=True)

merged_df = combine_df
merged_df['time'] = pd.to_datetime(merged_df['time'])
for i in range(len(df_collection)):
    print(i)
    df_collection[i]['time'] = pd.to_datetime(df_collection[i]['time'])
    #ValueError: Merge keys contain null values on left side
    
    print(len(merged_df['time'])-merged_df['time'].count())
    print(len(merged_df['time'])-df_collection[i]['time'].count())
    merged_df = pd.merge_asof(merged_df,df_collection[i],on='time',direction='nearest')
merged_df = merged_df.set_index('time')
daily_df = merged_df.resample('1D').mean()#1天数据
three_hour_df = merged_df.resample('3H').mean()#3小时数据
half_hour_df = merged_df.resample('30min').mean()#半小时数据

hetero_collection = []

67
0
1
56261


ValueError: Merge keys contain null values on left side

In [35]:
from data_upload import data_pretreatment
df = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\dataframe4.csv")
data_pretreatment(4,df)

plants = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\Plant_data_halfhour.csv")
whole_time=[pd.to_datetime(day+' '+hour) for day,hour in zip(plants['Date'],plants['Time'])]
plants['time']=whole_time
plants['pixel diff']=plants['Area per plant'].diff(1)
plants=plants.set_index('time')
plants['time'] = plants.index
#plants = plants[:-1]
re_arranged_plants = pd.DataFrame({})
df_collection = []

for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])
    #new_df = new_df.set_index('time')
    #new_df.index = pd.to_datetime(new_df.index)
    #ew_df = new_df.resample('30m').mean()
    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass
from glob import glob
import os 

dataframes4 = {}
csv_path = "arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)

for i in range(num_files):
    dataframes4['df'+str(i)] = pd.read_csv("./arranged_data4/sub_df"+str(i)+".csv")

combine_df = pd.concat([dataframes4[key] for key in dataframes4.keys()],ignore_index=True)
#for i in range(11,33):
#    combine_df=pd.concat([combine_df,dataframes4['df'+str(i)]],axis=1)

merged_df = combine_df
merged_df['time'] = pd.to_datetime(merged_df['time'])
for i in range(len(df_collection)):
    print(i)
    df_collection[i]['time'] = pd.to_datetime(df_collection[i]['time'])
    merged_df = pd.merge_asof(merged_df,df_collection[i],on='time',direction='nearest')

0
1
2
3
4
5
6
7
8


In [37]:
merged_df

,Unnamed: 0.1,Unnamed: 0,空调光期温度设定,空调暗期温度设定,空调温度上限设定,空调温度下限设定,新风机手动开度,新风机手自动模式,空调自动模式,户外温度,...,plant 5,plant 5 diff,plant 6,plant 6 diff,plant 7,plant 7 diff,plant 8,plant 8 diff,plant 9,plant 9 diff
0,0,0,22.0,18.0,18.0,17.0,100.0,False,False,17.925346,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
1,1,1,22.0,18.0,18.0,17.0,100.0,False,False,17.170139,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
2,2,2,22.0,18.0,18.0,17.0,100.0,False,False,17.378471,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
3,3,3,22.0,18.0,18.0,17.0,100.0,False,False,17.635998,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
4,4,4,22.0,18.0,18.0,17.0,100.0,False,False,17.774883,...,107.47,NaN,NaN,NaN,162.56,NaN,137.80,NaN,139.15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35716,35716,35716,24.5,21.6,19.5,18.5,0.0,False,True,24.652779,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35717,35717,35717,24.5,21.6,19.5,18.5,0.0,False,True,24.537041,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35718,35718,35718,24.5,21.6,19.5,18.5,0.0,False,True,24.635414,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
35719,35719,35719,24.5,21.6,19.5,18.5,0.0,False,True,24.733795,...,7.20,-0.3,NaN,NaN,224.53,0.02,8.65,-1.42,179.66,-5.86
